In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import subprocess

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 71.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=3b6382c9aceb3c31035ccc0fe2b50103f9fad195683eed24e7a31c4018b063ef
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib
japanize_matplotlib has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 22.5 MB/s eta 0:00:00
catboost has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00
optuna has been installed.
geolib has been installed.
Directory and file for OpenCL vendors have been created.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns
import pickle
from catboost import CatBoostRegressor, Pool
import os

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

from utils import (
                   to_cat_col,
                   VotingModel,
                   check_dataframe_elements
                   )

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

model_cat_lq_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_cat_lq_cv_20240908_141325_0_06609571831620502.pkl"
model_lgbm_l1_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_20240908_110735_0_06585289841.pkl"
#model_lgbm_l1_cv_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_sub_sub_0_06585289841760715.pkl"
model_lgbm_l1_cv_target_path="/content/drive/MyDrive/yuto/project_directory/models/model_lgbm_l1_cv_target_20240910_144521_0_06636916360273092.pkl"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
def visualize_hyperparameters(estimators):
    if isinstance(estimators, VotingModel):  # 入力がVotingModelかどうかを確認します
        for estimator in estimators.estimators:  # VotingModel内の推定器を反復処理します
            params = estimator.get_params()
            print(f"Estimator: {estimator.__class__.__name__}")  # 推定器のタイプを表示します
            #for param, value in params.items():
            #    print(f"  {param}: {value}")
            print(params)
    else:
        params = estimators.get_params()
        print(f"Estimator: {estimators.__class__.__name__}")  # 推定器のタイプを表示します
        #for param, value in params.items():
        #    print(f"  {param}: {value}")
        print(params)

# モデルの読み込み
def read_model(model_path):
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    return model

In [5]:
df=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv",index_col=0)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '単位面積あたりの取引価格_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]
#df=df.drop(columns=["住所_地区名欠損フラグ","address","title",TOTAL_PRICE_LOG],axis=1)

In [6]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

#print("Train DataFrame:")
#print(df_train)
#print("\nTest DataFrame:")
#print(df_test)

In [7]:
#check_dataframe_elements(df)

In [8]:
def decode_price(y_pred,df_test):
    # 単位面積あたりの取引価格の対数から10のべき乗を取る
    price_per_area = 10**y_pred_
    # その値に面積を掛けて、取引価格（総額）を計算
    predicted_price_total = price_per_area * df_test["面積（㎡）"]
    # 取引価格（総額）の対数を計算
    y_pred= np.log10(predicted_price_total)
    #y_pred=y_pred.rename(columns={"面積（㎡）":"取引価格（総額）_log"})
    y_pred.name = "取引価格（総額）_log"
    return y_pred

In [9]:
#print(abc)

Catboost l1

In [10]:
# モデルを格納するリスト
models = []

# 各foldのモデルを読み込む
for fold in range(1, 9):
    print(f"modelの読み込み -> fold{fold}開始")
    model_path = os.path.join("/content/drive/MyDrive/yuto/project_directory/models/", f'v1model_cat_lq_fold_{fold}.cbm')
    model = CatBoostRegressor()
    model.load_model(model_path)
    models.append(model)
    print(f"modelの読み込み -> fold{fold}終了")

# ハイパーパラメータの表示
for fold, model in enumerate(models, start=1):
    print(f"Fold {fold}のハイパーパラメータ:")
    print(model.get_params())

model_cat=VotingModel(models,cat_cols)

y_pred_cat=pd.Series(model_cat.predict_cat(df_test.drop(f"{PRICE_PER_AREA_LOG}",axis=1)),index=df_test.index)
#y_pred_lgbm=pd.Series(model_lgbm.predict_lgbm(df_test.drop(f"{TARGET}",axis=1)),index=df_test.index)
#y_pred_xg=pd.Series(model_xg.predict_xg(df_test_tree.drop("取引価格（総額）_log",axis=1)),index=df_test_tree.index)

# 単位面積あたりの取引価格の対数から10のべき乗を取る
price_per_area = 10**y_pred_cat
# その値に面積を掛けて、取引価格（総額）を計算
predicted_price_total = price_per_area * df_test["面積（㎡）"]
# 取引価格（総額）の対数を計算
y_pred= np.log10(predicted_price_total)

#y_pred=y_pred_cat
#y_pred=y_pred_lgbm
# y_predをデータフレームに変換
df_pred = pd.DataFrame(y_pred, columns=["取引価格（総額）_log"])

# インデックスをリセットし、カラム名を'ID'に変更
#df_pred.reset_index(inplace=True)
#df_pred.rename(columns={'index': 'ID'}, inplace=True)

# 結果を表示
print(df_pred)

# 損失関数lqでトレーニングしたCatboostモデルを呼び出す
#model_cat_lq_cv=read_model(model_cat_lq_cv_path)
#visualize_hyperparameters(model_cat_lq_cv)

#y_pred_cat_lq_cv=pd.Series(model_cat_lq_cv.predict_cat(df_test.drop(TOTAL_PRICE_LOG,axis=1)),index=df_test.index)
#y_pred_cat=pd.Series(model_cat_lq_cv.predict_cat(df_test.drop(TOTAL_PRICE_LOG,axis=1)),index=df_test.index)
#y_pred_cat=decode_price(y_pred_cat,df_test)
# Seriesの名前を変更
#y_pred_cat.name = "取引価格（総額）_log"

# y_predをデータフレームに変換
#df_pred = pd.DataFrame(y_pred_cat, columns=["取引価格（総額）_log"])

# 結果を表示
#print(df_pred)

#df_pred.to_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/test_cat_cv.csv")

modelの読み込み -> fold1開始
modelの読み込み -> fold1終了
modelの読み込み -> fold2開始
modelの読み込み -> fold2終了
modelの読み込み -> fold3開始
modelの読み込み -> fold3終了
modelの読み込み -> fold4開始
modelの読み込み -> fold4終了
modelの読み込み -> fold5開始
modelの読み込み -> fold5終了
modelの読み込み -> fold6開始
modelの読み込み -> fold6終了
modelの読み込み -> fold7開始
modelの読み込み -> fold7終了
modelの読み込み -> fold8開始
modelの読み込み -> fold8終了
Fold 1のハイパーパラメータ:
{'verbose': 500, 'od_type': 'Iter', 'score_function': 'NewtonCosine', 'use_best_model': True, 'border_count': 226, 'min_data_in_leaf': 14, 'random_strength': 8.902514383, 'learning_rate': 0.02141285001, 'task_type': 'GPU', 'iterations': 16321, 'l2_leaf_reg': 1.108061859, 'od_wait': 150, 'depth': 12, 'leaf_estimation_backtracking': 'AnyImprovement', 'eval_metric': 'MAE', 'leaf_estimation_iterations': 7, 'loss_function': 'Lq:q=1.3631413553264038', 'best_model_min_trees': 2191}
Fold 2のハイパーパラメータ:
{'verbose': 500, 'od_type': 'Iter', 'score_function': 'NewtonCosine', 'use_best_model': True, 'border_count': 226, 'min_data_in_leaf

In [11]:
y_pred_cat=pd.Series(model_cat.predict_cat(df_test.drop(f"{PRICE_PER_AREA_LOG}",axis=1)),index=df_test.index)
#y_pred_lgbm=pd.Series(model_lgbm.predict_lgbm(df_test.drop(f"{TARGET}",axis=1)),index=df_test.index)
#y_pred_xg=pd.Series(model_xg.predict_xg(df_test_tree.drop("取引価格（総額）_log",axis=1)),index=df_test_tree.index)

# 単位面積あたりの取引価格の対数から10のべき乗を取る
price_per_area = 10**y_pred_cat
# その値に面積を掛けて、取引価格（総額）を計算
predicted_price_total = price_per_area * df_test["面積（㎡）"]
# 取引価格（総額）の対数を計算
y_pred= np.log10(predicted_price_total)

#y_pred=y_pred_cat
#y_pred=y_pred_lgbm
# y_predをデータフレームに変換
df_pred = pd.DataFrame(y_pred, columns=["取引価格（総額）_log"])

# インデックスをリセットし、カラム名を'ID'に変更
#df_pred.reset_index(inplace=True)
#df_pred.rename(columns={'index': 'ID'}, inplace=True)

# 結果を表示
print(df_pred)

          取引価格（総額）_log
ID                    
1000107       6.675571
1000109       7.296850
1000112       6.564507
1000114       6.433753
1000117       7.395544
...                ...
47036620      7.543099
47037115      7.499714
47037850      7.315738
47037855      7.550711
47037856      7.544750

[54402 rows x 1 columns]


In [12]:
df_sub=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/final_predictions/v1test_cat_cv20240919_111048.csv")
print(df_sub)

             ID  取引価格（総額）_log
0       1000107      6.675571
1       1000109      7.296850
2       1000112      6.564507
3       1000114      6.433753
4       1000117      7.395544
...         ...           ...
54397  47036620      7.543099
54398  47037115      7.499714
54399  47037850      7.315738
54400  47037855      7.550711
54401  47037856      7.544750

[54402 rows x 2 columns]


In [13]:
#del model_cat_lq_cv,df_pred

In [14]:
print(abc)

NameError: name 'abc' is not defined

LightGBM l1 CV

LightGBM l1 CV target